<a href="https://colab.research.google.com/github/gnsandeep/Fine-Tuning-BERT/blob/master/BERTSeq_W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Training and Test data to Pandas DataFrame


In [1]:
import pandas as pd
def load_training_data_to_pandas(filename = 'train.jsonl'):
    X = []
    Y = []
    fhand = open(filename,encoding='utf8')
    for line in fhand:
        data = json.loads(line)

        lt = data['context']
        lt.reverse()
        fullTweet =   data['response'] + " " + ''.join(lt)

        X.append(fullTweet)
        Y.append(data['label'])
 
    
    twitterdata = pd.DataFrame({'Tweets': X,'Labels': Y})

def load_test_data_to_pandas(filename = 'test.jsonl'):
    tid = []
    X = []
    Y = []
    fhand = open(filename,encoding='utf8')
    for line in fhand:
        data = json.loads(line)
        tid.append(data['id'])
        lt = data['context']
        lt.reverse()
        fullTweet =   data['response'] + " " + ''.join(lt)

        X.append(fullTweet)
        
    
    twitterdata_t = pd.DataFrame({'ID': tid,
                   'Tweets': X})

In [2]:
#load_training_data_to_pandas()
#load_test_data_to_pandas()

In [3]:
#import pandas as pd
twitterdata = pd.read_csv("dataPandas.csv")
twitterdata.tail()

,Tweets,Labels
4995,@USER You don't . I have purchased a lot on Am...,NOT_SARCASM
4996,@USER #Emotions you say 🤔 never knew that I th...,NOT_SARCASM
4997,"@USER @USER @USER You are so right ... "" Yes !...",NOT_SARCASM
4998,@USER @USER @USER Another lazy delusional vote...,NOT_SARCASM
4999,@USER @USER I hope you know no news outlet fro...,NOT_SARCASM


In [4]:
twitterdata_t = pd.read_csv("dftestdata.csv")
twitterdata_t.tail()

,ID,Tweets
1795,twitter_1796,@USER @USER @USER is definitely the best out t...
1796,twitter_1797,@USER @USER Ye let her out run wild and infect...
1797,twitter_1798,"@USER @USER @USER Thanks for that , I would ha..."
1798,twitter_1799,@USER @USER @USER Yes also #found this on #new...
1799,twitter_1800,@USER @USER @USER you still need to send the l...


Cleanup Training and Test Data

In [5]:
#remove URL
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'http(\S)+', r'')
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'http ...', r'')
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'http', r'')
twitterdata[twitterdata['Tweets'].str.contains(r'http')]
# remove RT, @
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
twitterdata[twitterdata['Tweets'].str.contains(r'RT[ ]?@')]
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'@[\S]+',r'')

#remove non-ascii words and characters
twitterdata['Tweets'] = [''.join([i if ord(i) < 128 else '' for i in text]) for text in twitterdata['Tweets']]
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'_[\S]?',r'')
#remove &, < and >
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'&amp;?',r'and')
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'&lt;',r'<')
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'&gt;',r'>')
# remove extra space
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'[ ]{2, }',r' ')
# insert space between punctuation marks
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')
# lower case and strip white spaces at both ends
twitterdata['Tweets'] = twitterdata['Tweets'].str.lower()
twitterdata['Tweets'] = twitterdata['Tweets'].str.strip()

In [6]:
#remove URL
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'http(\S)+', r'')
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'http ...', r'')
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'http', r'')
twitterdata_t[twitterdata_t['Tweets'].str.contains(r'http')]
# remove RT, @
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
twitterdata_t[twitterdata_t['Tweets'].str.contains(r'RT[ ]?@')]
#twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'@[\S]+',r'')​
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'@[\S]+',r'')

#remove non-ascii words and characters
twitterdata_t['Tweets'] = [''.join([i if ord(i) < 128 else '' for i in text]) for text in twitterdata_t['Tweets']]
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'_[\S]?',r'')
#remove &, < and >
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'&amp;?',r'and')
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'&lt;',r'<')
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'&gt;',r'>')
# remove extra space
twitterdata['Tweets'] = twitterdata['Tweets'].str.replace(r'[ ]{2, }',r' ')
# insert space between punctuation marks
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')
# lower case and strip white spaces at both ends
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.lower()
twitterdata_t['Tweets'] = twitterdata_t['Tweets'].str.strip()

In [7]:
twitterdata.loc[twitterdata['Labels'] == 'SARCASM', 'Labels'] = 1
twitterdata.loc[twitterdata['Labels'] == 'NOT_SARCASM', 'Labels'] = 0

In [8]:
sentences = twitterdata.Tweets.values
labels = twitterdata.Labels.values.astype(int)

In [9]:
sentences_t = twitterdata_t.Tweets.values


If GPU is available use GPU


In [10]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


Install Transformers

In [11]:
!pip install transformers==3.5.1

     |████████████████████████████████| 1.3MB 8.8MB/s 
     |████████████████████████████████| 1.1MB 30.0MB/s 
     |████████████████████████████████| 890kB 44.0MB/s 
     |████████████████████████████████| 2.9MB 24.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c9fd183706e4c451439ceec26dc74ccddaaeb5722e22765d91436258ebe2892e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


In [13]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  1314


In [14]:
max_len

1314

Takenize Training and Test data , BERT supports 512 Max Lenght , we have used 256

In [15]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
#print('Original: ', sentences[0])
#print('Token IDs:', input_ids[0])

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_t = []
attention_masks_t = []

# For every sentence...
for sent_t in sentences_t:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent_t,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_t.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_t.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_t = torch.cat(input_ids_t, dim=0)
attention_masks_t = torch.cat(attention_masks_t, dim=0)

# Print sentence 0, now as a list of IDs.
#print('Original: ', sentences_t[0])
#print('Token IDs:', input_ids_t[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training and Validation Split

In [16]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)
dataset_t = TensorDataset(input_ids_t, attention_masks_t)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,500 training samples
  500 validation samples


Create DataLoader for Training , Validation and Test Datasets

In [17]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

score_dataloader = DataLoader(
            dataset_t, # The validation samples.
            sampler = SequentialSampler(dataset_t), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

Load BertForSequenceClassification

In [18]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Set Model Hyper Parameters and Learning Rate Scheduler

In [19]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
'''no_decay = ['bias', 'LayerNorm.weight']

optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)'''

"no_decay = ['bias', 'LayerNorm.weight']\n\noptimizer_grouped_parameters = [\n    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},\n    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}\n]\noptimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)"

In [20]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 10

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Utility Function for Accuracy and Elapsed Time

In [21]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [22]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Training and Validation Loop

In [23]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 10 ========
Training...
  Batch    40  of    141.    Elapsed: 0:00:53.
  Batch    80  of    141.    Elapsed: 0:01:49.
  Batch   120  of    141.    Elapsed: 0:02:49.

  Average training loss: 0.53
  Training epcoh took: 0:03:19

Running Validation...
  Accuracy: 0.76
  Validation Loss: 0.50
  Validation took: 0:00:08

======== Epoch 2 / 10 ========
Training...
  Batch    40  of    141.    Elapsed: 0:00:59.
  Batch    80  of    141.    Elapsed: 0:01:58.
  Batch   120  of    141.    Elapsed: 0:02:57.

  Average training loss: 0.39
  Training epcoh took: 0:03:28

Running Validation...
  Accuracy: 0.81
  Validation Loss: 0.44
  Validation took: 0:00:08

======== Epoch 3 / 10 ========
Training...
  Batch    40  of    141.    Elapsed: 0:00:59.
  Batch    80  of    141.    Elapsed: 0:01:58.
  Batch   120  of    141.    Elapsed: 0:02:58.

  Average training loss: 0.26
  Training epcoh took: 0:03:28

Running Validation...
  Accuracy: 0.75
  Validation Loss: 0.65
  Validation 

TestData Set Loop 

In [24]:
# empty list to save the model predictions
score_total_preds = []

  # iterate over batches
for step,batch in enumerate(score_dataloader):
    
    # Progress update every 50 batches.
    if step % 10 == 0 and not step == 0:
      
        # Calculate elapsed time in minutes.
        #elapsed = format_time(time.time() - t0)
            
        # Report progress.
        print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(score_dataloader)))

    # push the batch to gpu
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    #batch = [t.to(device) for t in batch]

    #sent_id, mask = batch

    # deactivate autograd
    with torch.no_grad():
      
        # model predictions
        #preds = model(sent_id, mask)
        preds = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask)
        #print('Length of Preds :', len(preds))

        # compute the validation loss between actual and predicted values
        #loss = cross_entropy(preds,labels)

        #total_loss = total_loss + loss.item()
        #print(type(preds),preds)
        #preds = np.asarray(preds)
        preds = preds[0].detach().cpu().numpy()

        #print(type(preds),preds[0])
        preds = np.argmax(preds, axis = 1)


        score_total_preds.extend(preds.tolist())

  Batch    10  of     57.
  Batch    20  of     57.
  Batch    30  of     57.
  Batch    40  of     57.
  Batch    50  of     57.


In [25]:
len(score_total_preds)

1800

In [26]:
twitterdata_t['Predict'] = score_total_preds

In [27]:
twitterdata_t['PLabel'] = np.where(twitterdata_t['Predict'] == 1, "SARCASM", "NOT_SARCASM")

In [28]:
twitterdata_t.head()

,ID,Tweets,Predict,PLabel
0,twitter_1,"my 3 year old , that just finished reading nie...",0,NOT_SARCASM
1,twitter_2,how many verifiable lies has he told now ? 15 ...,1,SARCASM
2,twitter_3,maybe docs just a scrub of a coach ... i mean ...,1,SARCASM
3,twitter_4,is just a cover up for the real hate inside ....,0,NOT_SARCASM
4,twitter_5,the irony being that he even has to ask why . ...,1,SARCASM


In [29]:
twitterdata_t.to_csv('answer_BERTSeq.txt', columns = ["ID" , "PLabel"] , header = False , index = False)